In [ ]:
!pip install sentencepiece transformers datasets evaluate sacrebleu peft  git+https://github.com/huggingface/huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "Helsinki-NLP/opus-mt-sla-en"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cjvt/rsdo4_en_sl")

In [ ]:
reduced_train_dataset = dataset["train"].select(range(50000))
dataset = reduced_train_dataset.train_test_split(test_size=0.2)
dataset

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
tokenizer.src_lang = "slv"
tokenizer.tgt_lang = "eng"

In [ ]:
prefix = ""

def preprocess_function(examples):

    inputs = [prefix + example for example in examples["sl_seq"]]
    targets = [example for example in examples["en_seq"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [ ]:
tokenized_dataset = dataset.map(preprocess_function, remove_columns=['en_seq', 'sl_seq'], batched=True)
print(tokenized_dataset)
print(tokenized_dataset["train"][0])

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

In [ ]:
!pip install sacrebleu

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, BartConfig

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
from peft import get_peft_model, TaskType, LoHaConfig

loha_alpha = 32
loha_rank_dropout = 0.1
loha_module_dropout = 0.0
loha_r = 16

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=loha_r,
    alpha=loha_alpha,
    target_modules=["q_proj", "v_proj"],
    inference_mode=False,
    base_model_name_or_path=model_name
)


In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
new_model_name = "translation_slo_eng_opus-mt-sla-en_loha"

training_args = Seq2SeqTrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
model_location = "anzeo/" + new_model_name
trainer.push_to_hub(model_location)

In [ ]:
text = "Translate the following text from Slovenian to English: sončen dan "

from transformers import pipeline

translator = pipeline("translation_sl_to_en", model=model_location, max_length=400)
translator(text)